### Handling special token indices generated by GPT indexing into codebook and crashing inference
- In atypical setting GPT has no knowledge of how to model sequences and sometimes we can get unlucky and get special token indices like, start token or filler indices to optimize kernel blocks on GPUs with nice numbers into our generated sequence.
- These indices are invalid for decoder and quantizer to process and crashes inferences, to handle we guard the sampling by making sure that we only sample valid indices
- cat only if torch.all ((xcol >=0) & (xcol < quantizer.codebook.weight.size(0))

### One of the ways codebook surgeries on pretrained models is done to induce semantics in downstream tasks

In [22]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from gpt import GPT, GPTConfig

model = GPT (GPTConfig(vocab_size=8200)).to('cuda').eval()
checkpoint = torch.load("./logs/model_34000.pt")
model.load_state_dict(checkpoint['model'])


C:\Users\Zen\AppData\Local\Temp\ipykernel_23244\1375790070.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load("./logs/model_34000.pt")


<All keys matched successfully>

In [23]:
from resnet_vqgan import VQGan
vqgan = VQGan ().to('cuda').eval()


In [24]:
#  input to decoder
post_quant_activation = torch.randn (4, 1024, 16, 16).to('cuda')
image = vqgan.decoder(post_quant_activation)

In [25]:
image.shape, post_quant_activation.dtype

(torch.Size([4, 3, 256, 256]), torch.float32)

In [26]:
import numpy as np
def load_indices (filename, for_inference=True):
    npt = np.load (filename)
    npt = npt.astype (np.int32)
    ptt = torch.tensor (npt, dtype=torch.long)
    if for_inference:
        ptt = ptt[:,1:]
    return ptt

half_context_latent_tensor = load_indices ("./contextRenderLatentShard/inference_shard_01.npy", for_inference=False)
half_context_latent_tensor.shape, half_context_latent_tensor.dtype

(torch.Size([4, 257]), torch.int64)

In [27]:
half_context_latent_tensor = half_context_latent_tensor[:, :129]

In [28]:
half_context_indices = half_context_latent_tensor

In [29]:
xgen = half_context_latent_tensor.to('cuda')

with torch.no_grad():
    while xgen.size(1) < 257:
        with torch.autocast(device_type='cuda', dtype=torch.bfloat16):
            logits, loss = model (xgen)
        logits = logits[:, -1, :] # B, vocab_size
        probs = F.softmax (logits, dim=-1)
        topk_probs, topk_indices = torch.topk (probs, 50, -1)
        ix = torch.multinomial (topk_probs, num_samples=1)
        xcol = torch.gather (topk_indices, -1, ix)

        xgen = torch.cat ((xgen, xcol), dim=-1)
xgen.shape, xgen.dtype

(torch.Size([4, 257]), torch.int64)

In [30]:
xgen = xgen[:,1:]

In [31]:
xgen.shape

torch.Size([4, 256])

In [32]:
latent_vectors_tensor = vqgan.quantizer.codebook(xgen)

In [33]:
latent_vectors_tensor.shape, latent_vectors_tensor.dtype

(torch.Size([4, 256, 1024]), torch.float32)

In [34]:
latent_vectors_tensor = latent_vectors_tensor.view (4, 16, 16, 1024)

In [35]:
latent_vectors_tensor = latent_vectors_tensor.permute(0, 3, 1, 2).contiguous()

In [36]:
latent_vectors_tensor.shape

torch.Size([4, 1024, 16, 16])

In [37]:
post_quant_activation = vqgan.post_quant_conv(latent_vectors_tensor)

In [38]:
post_quant_activation.shape, post_quant_activation.dtype

(torch.Size([4, 1024, 16, 16]), torch.float32)

In [39]:
image = vqgan.decoder(post_quant_activation)

In [40]:
image.shape

torch.Size([4, 3, 256, 256])

In [41]:
image.dtype

torch.float32

In [3]:
import torch
import torch.nn.functional as F
topk_indices = torch.randn(10)
ix = torch.multinomial (F.softmax(torch.randn(10)), 1)

x_col = torch.gather (topk_indices, -1, ix)

C:\Users\Zen\AppData\Local\Temp\ipykernel_17188\2664473640.py:4: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  ix = torch.multinomial (F.softmax(torch.randn(10)), 1)


In [7]:
x_col.item()

1.0402305126190186

In [9]:
from resnet_vqgan import VQGan
model = VQGan()
model.quantizer.codebook.weight.size(0)

8192

In [20]:
x_col = torch.tensor([[4], [4], [4], [6]])
if torch.all((x_col >= 0) & (x_col < 5)):
    print ("deez nuts")


In [25]:
a = torch.ones (2,3)

In [26]:
a.shape

torch.Size([2, 3])

In [28]:
asquared = torch.sum(a**2, dim=1, keepdim=True)
asquared.shape

torch.Size([2, 1])

In [29]:
asquared

tensor([[3.],
        [3.]])